In [1]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

In [2]:
nnfs.init()

In [40]:
# Dense Layer
class Layer_Dense:

    # Layer init
    def __init__(self, n_inputs, n_neurons):
        # init wts and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))

    # Forward Pass
    def forward(self, inputs):
        # calulating the forward pass
        self.output = np.dot(inputs, self.weights) + self.biases

# ReLU Activation
class Activation_ReLU:

    # Forward Pass
    def forward(self, inputs):
        # ReLU gives max of 0, x. Inputs is the output of the first layer and np.maximum performs this for every value in inputs
        self.output = np.maximum(0, inputs)

# Softmax Activation
class Activation_Softmax:

    # Forward Pass
    def forward(self, inputs):
        # exp(x) / sum(exp(x))
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True)) #subtraction is done to ensure there is no explosion of bigger values
        probablitites = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        self.output = probablitites


# Loss

class Loss:
    # calculate loss from y and output
    def calculate(self, output, y):

        # apply forward function
        sample_losses = self.forward(output, y)
        print("Loss: ", sample_losses)

        # Calculate mean loss
        mean_loss = np.mean(sample_losses)

        return mean_loss
    
# Cross Entropy Loss
class Loss_CategoricalCrossEntropy(Loss):

    # Forward function
    def forward(self, y_pred, y_true):

        # No. of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by 0
        # Clip data from both sides to prevent change in mean
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)

        # Calculate probabilities
        # Only category labels
        if(len(y_true.shape) == 1):
            correct_confidences = y_pred_clipped[range(samples), y_true]

        # Y_true is one hot encoded
        if(len(y_true.shape) == 2):
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)

        # loss
        negative_log_likelihoods = -np.log(correct_confidences)

        return negative_log_likelihoods



In [41]:
X, y = spiral_data(samples=100, classes=3)

# Create a dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2, 3)

# Create the ReLU Actuvation 
activation1 = Activation_ReLU()

# Create the 2nd dense layer with 3 input features as the previous layer gave 3 outputs ans 3 output as we have 3 classes
dense2 = Layer_Dense(3, 3)

# Create the Softmax Activation
activation2 = Activation_Softmax()

# Create the loss function
loss_function= Loss_CategoricalCrossEntropy()


In [42]:
# Forward pass
dense1.forward(X)
# print(X.shape)
# Apply the first activation to the ouput of 1st layer
activation1.forward(dense1.output)
# print(dense1.output.shape)
# Forward pass to the 2nd layer
dense2.forward(activation1.output)

# Applying the 2nd activation
activation2.forward(dense2.output)

print(activation2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.33333334 0.3333333  0.3333333 ]
 [0.33333433 0.333333   0.33333272]
 [0.3333336  0.33333206 0.33333433]
 [0.33333465 0.3333328  0.3333325 ]]


In [43]:
loss = loss_function.calculate(activation2.output, y)
print(loss)

Loss:  [1.0986123 1.0986123 1.0986093 1.0986115 1.0986084 1.0986072 1.0986013
 1.0986123 1.098603  1.0985992 1.0986001 1.0985905 1.0985931 1.0985918
 1.0985854 1.0985839 1.0985813 1.0985827 1.0985768 1.0985798 1.0985814
 1.0985931 1.0985707 1.0985677 1.0985938 1.0985674 1.0986011 1.0986075
 1.0985999 1.0985847 1.098599  1.0986055 1.098607  1.0985966 1.0985916
 1.0985959 1.0986013 1.0986038 1.098576  1.0986006 1.0985991 1.0985937
 1.0985923 1.098598  1.0985928 1.0985994 1.0985904 1.0986005 1.0985914
 1.0986089 1.0986013 1.0985883 1.098594  1.0986071 1.0985973 1.0985985
 1.0986123 1.098595  1.0986062 1.0986123 1.0986067 1.0986123 1.0985811
 1.0985891 1.0986114 1.0986099 1.0986123 1.0985636 1.0985603 1.0985905
 1.0985609 1.0986097 1.0984857 1.0984936 1.0985482 1.0984738 1.0984726
 1.0984826 1.0984821 1.0984572 1.0984572 1.0984572 1.0985271 1.0984677
 1.0984459 1.0984439 1.0984418 1.0984397 1.0985104 1.0984614 1.0985291
 1.0984967 1.0985051 1.0985436 1.0985863 1.0985788 1.0985844 1.0985649